<a href="https://colab.research.google.com/github/asokraju/CDPRgym/blob/main/CDPR_rlzoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RL Baselines3 Zoo: Training in Colab



Github Repo: [https://github.com/DLR-RM/rl-baselines3-zoo](https://github.com/DLR-RM/rl-baselines3-zoo)

Stable-Baselines3 Repo: [https://github.com/DLR-RM/rl-baselines3-zoo](https://github.com/DLR-RM/stable-baselines3)


# Install Dependencies



In [1]:
!apt-get install swig cmake ffmpeg freeglut3-dev xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.11).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


## Clone RL Baselines3 Zoo Repo

In [2]:
!git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo

Cloning into 'rl-baselines3-zoo'...
remote: Enumerating objects: 4099, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 4099 (delta 15), reused 27 (delta 7), pack-reused 4049
Receiving objects: 100% (4099/4099), 2.81 MiB | 2.54 MiB/s, done.
Resolving deltas: 100% (2657/2657), done.
Submodule 'rl-trained-agents' (https://github.com/DLR-RM/rl-trained-agents) registered for path 'rl-trained-agents'
Cloning into '/content/rl-baselines3-zoo/rl-trained-agents'...
remote: Enumerating objects: 2231, done.        
remote: Counting objects: 100% (74/74), done.        
remote: Compressing objects: 100% (54/54), done.        
remote: Total 2231 (delta 20), reused 68 (delta 19), pack-reused 2157
Receiving objects: 100% (2231/2231), 1.35 GiB | 17.52 MiB/s, done.
Resolving deltas: 100% (489/489), done.
Submodule path 'rl-trained-agents': checked out '1e2a45e5d06efd6cc15da6cf2d1939d72dcbdf87'


In [2]:
%cd /content/rl-baselines3-zoo/

/content/rl-baselines3-zoo


In [4]:
!git clone https://github.com/asokraju/CDPRgym.git

Cloning into 'CDPRgym'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 2), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


### Install pip dependencies

In [ ]:
!pip install -r requirements.txt

## Train an RL Agent


The train agent can be found in the `logs/` folder.

Here we will train A2C on CartPole-v1 environment for 100 000 steps. 


To train it on Pong (Atari), you just have to pass `--env PongNoFrameskip-v4`

Note: You need to update `hyperparams/algo.yml` to support new environments. You can access it in the side panel of Google Colab. (see https://stackoverflow.com/questions/46986398/import-data-into-google-colaboratory)

In [ ]:
!python train.py --algo ppo --env CDPR-v0 --n-timesteps 100000 --vec-env "subproc"

========== CDPR-v0 ==========
Seed: 2307525420
Loading hyperparameters from: hyperparams/ppo.yml
Default hyperparameters for environment (ones being tuned will be overridden):
OrderedDict([('clip_range', 0.2),
             ('ent_coef', 0.0),
             ('gae_lambda', 0.95),
             ('gamma', 0.9),
             ('learning_rate', 0.001),
             ('n_envs', 20),
             ('n_epochs', 10),
             ('n_steps', 20000),
             ('n_timesteps', 10000000.0),
             ('policy', 'MlpPolicy'),
             ('sde_sample_freq', 4),
             ('use_sde', True),
             ('vec_env_wrapper',
              ['stable_baselines3.common.vec_env.VecMonitor',
               'stable_baselines3.common.vec_env.vec_normalize.VecNormalize'])])
Using 20 environments
Overwriting n_timesteps with n=100000
Creating test environment
/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:216: UserWarning: WARN: Overriding environment CDPR-v0
  logger.warn("Overriding enviro

# Others Not Implemented

#### Evaluate trained agent


You can remove the `--folder logs/` to evaluate pretrained agent.

In [ ]:
!python enjoy.py --algo a2c --env CartPole-v1 --no-render --n-timesteps 5000 --folder logs/

#### Tune Hyperparameters

We use [Optuna](https://optuna.org/) for optimizing the hyperparameters.

Tune the hyperparameters for PPO, using a tpe sampler and median pruner, 2 parallels jobs,
with a budget of 1000 trials and a maximum of 50000 steps

In [ ]:
!python train.py --algo ppo --env MountainCar-v0 -n 50000 -optimize --n-trials 1000 --n-jobs 2 --sampler tpe --pruner median

### Record  a Video

In [ ]:
# Set up display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
!python -m utils.record_video --algo a2c --env CartPole-v1 --exp-id 0 -f logs/ -n 1000

### Continue Training

Here, we will continue training of the previous model

In [ ]:
!python train.py --algo a2c --env CartPole-v1 --n-timesteps 50000 -i logs/a2c/CartPole-v1_1/CartPole-v1.zip

In [ ]:
!python enjoy.py --algo a2c --env CartPole-v1 --no-render --n-timesteps 1000 --folder logs/